In [1]:
import pandas as pd
import seaborn as sns
import sys
import string
print('Original sys.path:', sys.path)

# Append a new directory to sys.path
sys.path.append('C:/Users/joepa/OneDrive/Desktop/computerScienceYear4/Data Mining/summary_eval/summary_eval')

# Print the updated sys.path
print('Updated sys.path:', sys.path)
from summary_eval.data import summary_df, prompts_df
from summary_eval.settings import TRAIN_SIZE
from summary_eval.testing import cross_validate

merged_df = pd.merge(summary_df, prompts_df, on='prompt_id')

2024-03-17 14:24:10,155 - INFO - Read 7165 summaries from C:\Users\joepa\OneDrive\Desktop\computerScienceYear4\Data Mining\summary_eval\data\summaries_train.csv
2024-03-17 14:24:10,160 - INFO - Read 4 prompts from C:\Users\joepa\OneDrive\Desktop\computerScienceYear4\Data Mining\summary_eval\data\prompts_train.csv


Original sys.path: ['c:\\Users\\joepa\\OneDrive\\Desktop\\computerScienceYear4\\Data Mining\\summary_eval\\notebooks', 'C:\\Users\\joepa\\OneDrive\\Desktop\\computerScienceYear4\\Data Mining\\summary_eval', 'C:\\Users\\joepa\\anaconda3\\python39.zip', 'C:\\Users\\joepa\\anaconda3\\DLLs', 'C:\\Users\\joepa\\anaconda3\\lib', 'C:\\Users\\joepa\\anaconda3', 'c:\\Users\\joepa\\OneDrive\\Desktop\\computerScienceYear4\\Data Mining\\summary_eval\\venv', '', 'c:\\Users\\joepa\\OneDrive\\Desktop\\computerScienceYear4\\Data Mining\\summary_eval\\venv\\lib\\site-packages', 'c:\\Users\\joepa\\OneDrive\\Desktop\\computerScienceYear4\\Data Mining\\summary_eval\\venv\\lib\\site-packages\\win32', 'c:\\Users\\joepa\\OneDrive\\Desktop\\computerScienceYear4\\Data Mining\\summary_eval\\venv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\joepa\\OneDrive\\Desktop\\computerScienceYear4\\Data Mining\\summary_eval\\venv\\lib\\site-packages\\Pythonwin']
Updated sys.path: ['c:\\Users\\joepa\\OneDrive\\Desktop\\com

In [2]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Generate BERT embeddings
def generate_bert_embeddings(text):
    with torch.no_grad():
        # Tokenize the text
        tokens = tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True)
        
        # Ensure that the tokenized sequence is not longer than 512 tokens
        tokens = tokens[:512]

        # Pad the sequence if it's shorter than 512 tokens
        if len(tokens) < 512:
            tokens += [0] * (512 - len(tokens))
        
        # Convert tokens to tensor
        tokens_tensor = torch.tensor([tokens])
        
        # Get BERT outputs
        outputs = bert_model(tokens_tensor)
        
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()  # Average pooling over all tokens


merged_df = merged_df.head(100)

# Apply function to 'text', 'prompt_text', and 'prompt_question' columns
merged_df['text_embeddings'] = merged_df['text'].apply(generate_bert_embeddings)
merged_df['prompt_embeddings'] = merged_df['prompt_text'].apply(generate_bert_embeddings)
merged_df['prompt_question_embeddings'] = merged_df['prompt_question'].apply(generate_bert_embeddings)


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [3]:
y = merged_df[['content', 'wording']]

X = merged_df[['text_embeddings', 'prompt_embeddings','prompt_question_embeddings']]
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = MultiOutputRegressor(HistGradientBoostingRegressor(random_state=0))

results_df = cross_validate(model, X_train, y_train)
results_df

2024-03-17 14:33:09,147 - INFO - Using 10x10 cross validation


  0%|          | 0/100 [00:00<?, ?it/s]

ValueError: setting an array element with a sequence.